In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
# from selenium.webdriver.support.expected_conditions import staleness_of
import pandas as pd 
import time
import re
import os
import requests

In [103]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_12_1link\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    
    # ====================================================================================================================
    # get the username tab
    try:
        provided_u_name = "EG78TP9"
        user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//input[@placeholder="Username"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//input[@placeholder="Password"]')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/input[@value="Login"]')))
        if check:
            driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
        
    except Exception as e:
        print(f"No check tab found and the error is {e}") 
    # ====================================================================================================================
    
    results = []
    
    # while True:
    details= {}
    # car name--> done
    try:
        car_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-jKVCRD fRxzLv"]')))
        if car_name:
            details['Title'] = car_name.text.strip()
        else:
            details['Title'] = "na"
    except Exception as e:
        print("No car name")
    
    
    
    # image card --> done   
    try:
        img_clk = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/div[contains(@class, "gallery__image")]')))
        img_clk.click()
        time.sleep(2)  # Small delay to allow modal to load

        imgs = []
        while True:
            try:
                img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[contains(@class, "modal-gallery__body")]')))
                img = img_card.find_element(By.TAG_NAME, "img")
                imgs.append(img.get_attribute("src"))

                # Locate and click the 'Next' button
                next_butn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[contains(@class, "hRVyLw")]')))
                next_butn.click()
                time.sleep(1)  # Delay to allow image change

            except Exception:
                print("No more images or error clicking next button.")
                break  # Exit loop when no next button

        # Close the image modal
        try:
            close = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[text()="Close"]')))
            time.sleep(1)
            close.click()
        except Exception as e:
            print("Error closing image modal:")

        # Store image links
        details['Images'] = ", ".join(imgs)
    except Exception as e:
        print("Error processing image card:")
        
        
    # bid pricing --> done
    try:
       pr_main_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-banner__purchase-block"]')))
       if pr_main_card:
            prices = WebDriverWait(pr_main_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="sc-kGXeez bMxRxo"]')))
            if prices:
                for price in prices:
                    pr_lbl = price.find_element(By.XPATH, './/label[@class="sc-gzVnrw kiynVB pricing-block__header"]').text.strip()
                    pr_val = price.find_element(By.XPATH, './/p[contains(@class, "sc-kAzzGY")]').text.strip()
                    if pr_lbl and pr_val:
                        details[pr_lbl] = pr_val
    except Exception as e:
        print(f"No pricings and error is {e}")
            
    
    # getting tabss information
    try:
        tabs_clck_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-header-block"]')))
        if tabs_clck_card:
        
            # get the veh details --> done
            try:
                veh_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Vehicle Details"]')))
                if veh_det_clck:
                    
                    driver.execute_script("arguments[0].scrollIntoView();", veh_det_clck)
                    veh_det_clck.click()
                    
                    # veh det informatin --> done
                    try:
                        veh_info_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details-top-block"]')))
                        if veh_info_card:
                            veh_rows = WebDriverWait(veh_info_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vehicle-details-detail-list__row"]')))
                            if veh_rows:
                                for veh_row in veh_rows:
                                    veh_lbl = veh_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[0].text.strip()
                                    veh_val = veh_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[1].text.strip()
                                    if veh_lbl and veh_val:
                                        details[veh_lbl] = veh_val
                                    else:
                                        print("No veh label and vale")
                            else:
                                print("No veh rows")
                        else:
                            print("No veh det card")
                    except Exception as e:
                        print("no veh det found")
                        
                    # specifictaion --> done
                    try:
                        spec_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-elJkPf bDeocW"]')))
                        if spec_card:
                            spec_btn = WebDriverWait(spec_card, 10).until(EC.presence_of_element_located((By.XPATH, './/button[@class="sc-hmzhuo hOMyGw accordion__header"]')))
                            if spec_btn:
                                time.sleep(1)
                                spec_btn.click()
                                
                                spec_det_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="accordion__content"]'))) 
                                if spec_card:
                                    spec_rows = WebDriverWait(spec_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="technical-spec-detail-list__row"]')))
                                    if spec_rows:
                                        for spec_row in spec_rows:
                                            spec_lbl = spec_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[0].text.strip()
                                            spec_val = spec_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[1].text.strip()
                                            if spec_lbl and spec_val:
                                                details[spec_lbl] = spec_val
                                            else:
                                                print("No spec label and vale")
                                    else:
                                        print("No spec rows")
                                else:
                                    print("No spec card")
                            else:
                                print("No spec button")
                        else:
                            print("No spec click")
                    except Exception as e:
                        print(" No spec card found")
                else:
                    print("No veh det card to click")
            except Exception as e:
                print("No veh card")
            
                
            # Standard Equipment --> done
            try:
                stnd_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Standard Equipment"]')))
                if stnd_det_clck:
                    driver.execute_script("arguments[0].scrollIntoView();", stnd_det_clck)
                    stnd_det_clck.click()

                    stnd_info_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                    if stnd_info_card:
                        stnd_rows = WebDriverWait(stnd_info_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="text-list__row"]')))
                        if stnd_rows:
                            for stnd_row in stnd_rows:
                                stnd_lbl = stnd_row.find_element(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]').text.strip()
                                stnd_vals = stnd_row.find_elements(By.TAG_NAME, 'li')
                                if stnd_vals:
                                    stnd_val_lst = [li.text.strip() for li in stnd_vals]
                                    if stnd_lbl and stnd_val_lst:
                                        details[stnd_lbl] = stnd_val_lst
                                    else:
                                        print("No standard labels and values")
                                        
                        else:
                            print("No standard rows")
                    else:
                        print("No standard card")
                else:
                    print("No veh det card to click")
            except Exception as e:
                print(f"No standard card")
            
            # Inspection Report --> done
            try:
                inspec_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Inspection Report"]')))
                if inspec_det_clck:
                    driver.execute_script("arguments[0].scrollIntoView();", inspec_det_clck)
                    inspec_det_clck.click()

                    inspec_card =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                    if inspec_card:
                        inspec_link = WebDriverWait(inspec_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Download PDF"]')))
                        if inspec_link:
                            inspec = inspec_link.get_attribute("href")
                            details['Inspection_report'] = inspec
                        else:
                            details['Inspection_report'] = "na"
                    else:
                        print("No inspec card")
                else:
                    print("No inspec card to click")
            except Exception as e:
                print(f"No inspec card and error is {e}")
                
            # Other Documents --> done
            try:
                other_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Other Documents"]')))
                if other_det_clck:
                    driver.execute_script("arguments[0].scrollIntoView();", other_det_clck)
                    other_det_clck.click()

                    other_card =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                    if other_card:
                        other_link = WebDriverWait(other_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="downloadBtn gaEvent-download"]')))
                        if other_link:
                            other = other_link.get_attribute("href")
                            details['Other Documents'] = other
                        else:
                            details['Other Documents'] = "na"
                    else:
                        print("No Other Documents card")
                else:
                    print("No Other Documents to click")
            except Exception as e:
                print(f"No Other Documents and error is {e}")
            
    except Exception as e:
        print("No tabs to click")
    
    results.append(details)
        
        # next button
        # try:
        #     next_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-bnXvFD kEdhnn"]'))) 
        #     next_btn =WebDriverWait(next_card, 10).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Next"]'))) 
        #     if next_btn:
        #         driver.execute_script("arguments[0].scrollIntoView();", next_btn)
        #         time.sleep(2)
        #         next_btn.click()
        #     else:
        #         print("No next button link")
        #         break
        # except Exception as e:
        #     print("No next button link")  
            
    
    
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("1link.csv", index=False)
    # time.sleep(2)
    driver.quit()
path = "https://disposalnetwork.1link.co.uk/uk/tb/app/vehicle/25261980"
scrape(path)

In [104]:
df =pd.read_csv("1link.csv")
df

,Title,STARTING BID,BUY NOW
0,MG MOTOR UK ZS\nELECTRIC HATCHBACK 130kW SE EV...,"£7,000","£9,000"
